In [ ]:
import mydemosaicking_header as mh
from mydensenet import DenseNet 

#read data
train_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/train/gt/'
train_train_path = 'E:/data/DIV2K/DIV2K_train_patch/train/train/'
valid_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/gt/'
valid_valid_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/valid/'
train_gt_patch = []
train_train_patch = []
valid_gt_img = []
valid_valid_img = []
rgb = ['r','g','b']

for i in rgb:
    gt_patch = '_gt_patch_all.h5'
    train_patch = '_train_patch_all.h5'
    gt_filename = i+gt_patch
    train_filename = i+train_patch
    temp = mh.load_h5(train_gt_path, gt_filename, 1)
    train_gt_patch.append(temp)
    temp = mh.load_h5(train_train_path, train_filename, 1)
    train_train_patch.append(temp)
    for j in range(1,101):
        valid_img = '_valid_img_'
        gt_img = '_gt_img_'
        h5 = '.h5'
        valid_filename = str(j) + valid_img + i + h5
        gt_filename = str(j) + gt_img + i + h5
        temp = mh.load_h5(valid_valid_path, valid_filename, j)
        valid_valid_img.append(temp)
        temp = mh.load_h5(valid_gt_path, gt_filename, j)
        valid_gt_img.append(temp)

In [ ]:
BATCH_SIZE = 64
EPOCHS = 100

model = DenseNet(nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.0, weights_path=None)

adam = mh.Adam(lr=0.0001)
model.compile(adam, loss='mse', metrics=[mh.PSNR, "accuracy"])
model.summary()

In [ ]:
r_filepath="./checkpoints/red/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
r_checkpoint = mh.ModelCheckpoint(r_filepath, monitor=mh.PSNR, verbose=1, mode='max')
r_callbacks_list = [r_checkpoint]

g_filepath="./checkpoints/green/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
g_checkpoint = mh.ModelCheckpoint(g_filepath, monitor=mh.PSNR, verbose=1, mode='max')
g_callbacks_list = [g_checkpoint]

b_filepath="./checkpoints/blue/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
b_checkpoint = mh.ModelCheckpoint(b_filepath, monitor=mh.PSNR, verbose=1, mode='max')
b_callbacks_list = [b_checkpoint]

model.fit_generator(mh.train_image_gen(train_train_patch, train_gt_patch, 0, BATCH_SIZE), steps_per_epoch=522939 // BATCH_SIZE,  \
					validation_data=mh.valid_image_gen(valid_valid_img, valid_gt_img, 100), validation_steps=100, \
					epochs=EPOCHS, workers=8, callbacks=r_callbacks_list)

#model.fit_generator(image_gen(g_train_list, TRAIN_BATCH_SIZE), steps_per_epoch=len(g_train_list) // TRAIN_BATCH_SIZE,  \
#					validation_data=image_gen(g_test_list, TEST_BATCH_SIZE), validation_steps=len(g_test_list) // TEST_BATCH_SIZE, \
#					epochs=EPOCHS, workers=8, callbacks=g_callbacks_list)

#model.fit_generator(image_gen(b_train_list, TRAIN_BATCH_SIZE), steps_per_epoch=len(b_train_list) // TRAIN_BATCH_SIZE,  \
#					validation_data=image_gen(b_test_list, TEST_BATCH_SIZE), validation_steps=len(b_test_list) // TEST_BATCH_SIZE, \
#					epochs=EPOCHS, workers=8, callbacks=b_callbacks_list)

print("Done training!!!")

print("Saving the final model ...")

#model.save('vdsr_model.h5')  # creates a HDF5 file 
del model  # deletes the existing model